In [ ]:
'''
This script contains code for SXGbgFold model
'''
# import libraries
import glob
import random
import numpy as np
import pandas as pd
import csv
import h5py
import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Layer, Dense, Dropout, LSTM, GRU, Conv1D, Conv2D, Conv2DTranspose, MaxPooling2D, AveragePooling2D, UpSampling2D
from tensorflow.keras.layers import concatenate, GlobalMaxPooling1D, Flatten, BatchNormalization
from tensorflow.keras.layers import Activation, Reshape, TimeDistributed, Embedding, Input
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adamax, Adadelta, Adagrad, Nadam
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l1, l2
from sklearn.model_selection import KFold, StratifiedKFold
from tqdm import tqdm

import parse_files as p
from features import bigram_features0, bigram_features1, bigram_features2, bigram_features3, bigram_features4, bigram_features5

from numpy.random import seed
from tensorflow.python.keras.backend import set_session
from keras import regularizers

In [ ]:
clsfeature = False # True/False (True if you want to use class features. Use only when predtype is Fold)
rawdata = 'hmm' # hmm/pssm
predtype = 'Fold' # Class/Fold
dataset = 'SCOPe' # dd/edd/tg/SCOPe/25_SCOPe_DDEDDTG

In [ ]:
# set seed
seed = 420
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

# initialize variables to store data and labels
labels = []
hmm = []
pssm = []
seq = []
seqlen = 200
biGram_features0 = []
biGram_features1 = []
biGram_features2 = []
biGram_features3 = []
biGram_features4 = []
biGram_features5 = []


# Load all the filenames of PSSM's
filelist = glob.glob('./data/'+dataset+'/'+rawdata+'/*.txt')

# Read all the labels of the given dataset
if dataset == "SCOPe":
	label_for_seq = pd.read_csv("./astral_2_08_final.csv") # Make sure all the sequences are in uppercase
else:
	label_for_seq = p.load_labels('./data/'+dataset+'_'+predtype+'_labels.txt')

# Read all the HMM and PSSM matrices of the given dataset
for i in range(0, len(filelist)):
	# HMM data
	if(rawdata == 'hmm'): # HMM data
		seq_hmm,prob_hmm,extras_hmm = p.parse_hmm(filelist[i]) # Parse HMM file
		tempseq = seq_hmm.upper() # Convert sequence to uppercase
		seq.append(tempseq) # Append sequence to seq list
		# Append label to labels list
		if dataset == "SCOPe":
			labels.append(label_for_seq.loc[label_for_seq["sequence"] == tempseq]["fold"].values[0])
		else:
			labels.append(label_for_seq[seq_hmm.upper()])
		if(clsfeature): # If class features are to be used
			# Append bigram features and class label to biGram_features list
			biGram_features0.append(((np.append((bigram_features0(prob_hmm)), (p.get_class_label(dataset, label_for_seq[tempseq])))).reshape(1, -1)))
			biGram_features1.append(((np.append((bigram_features1(prob_hmm)), (p.get_class_label(dataset, label_for_seq[tempseq])))).reshape(1, -1)))
			biGram_features2.append(((np.append((bigram_features2(prob_hmm)), (p.get_class_label(dataset, label_for_seq[tempseq])))).reshape(1, -1)))
			biGram_features3.append(((np.append((bigram_features3(prob_hmm)), (p.get_class_label(dataset, label_for_seq[tempseq])))).reshape(1, -1)))
			biGram_features4.append(((np.append((bigram_features4(prob_hmm)), (p.get_class_label(dataset, label_for_seq[tempseq])))).reshape(1, -1)))
			biGram_features5.append(((np.append((bigram_features5(prob_hmm)), (p.get_class_label(dataset, label_for_seq[tempseq])))).reshape(1, -1)))
		else: # If class features are not to be used
			# Append bigram features to biGram_features list
			biGram_features0.append(bigram_features0(prob_hmm))
			biGram_features2.append(bigram_features2(prob_hmm))
			biGram_features3.append(bigram_features3(prob_hmm))
			biGram_features4.append(bigram_features4(prob_hmm))
			biGram_features1.append(bigram_features1(prob_hmm))
			biGram_features5.append(bigram_features5(prob_hmm))

		norm_hmm = prob_hmm + 0.01
		if(len(norm_hmm) < seqlen): # If length of HMM data is less than seqlen, pad it with zeros
			for j in range(seqlen-len(norm_hmm)):
				norm_hmm = np.concatenate((norm_hmm,norm_hmm[0]*0))
		else: # If length of HMM data is greater than seqlen, truncate it
			norm_hmm = norm_hmm[:seqlen]
		hmm.append(norm_hmm) # Append HMM data to hmm list

	# PSSM data
	else:  
		seq_pssm,prob_pssm,lprob_pssm,extra_pssm = p.parse_pssm(filelist[i]) # Parse PSSM file
		tempseq = seq_pssm.upper() # Convert sequence to uppercase
		seq.append(tempseq) # Append sequence to seq list
		labels.append(label_for_seq[seq_pssm.upper()]) # Append label to labels list
		if(clsfeature): # If class features are to be used
			biGram_features0.append(((np.append((bigram_features0(prob_pssm)), (p.get_class_label(dataset, label_for_seq[tempseq])))).reshape(1, -1)))
			biGram_features1.append(((np.append((bigram_features1(prob_pssm)), (p.get_class_label(dataset, label_for_seq[tempseq])))).reshape(1, -1)))
			biGram_features2.append(((np.append((bigram_features2(prob_pssm)), (p.get_class_label(dataset, label_for_seq[tempseq])))).reshape(1, -1)))
			biGram_features3.append(((np.append((bigram_features3(prob_pssm)), (p.get_class_label(dataset, label_for_seq[tempseq])))).reshape(1, -1)))
			biGram_features4.append(((np.append((bigram_features4(prob_pssm)), (p.get_class_label(dataset, label_for_seq[tempseq])))).reshape(1, -1)))
			biGram_features5.append(((np.append((bigram_features5(prob_pssm)), (p.get_class_label(dataset, label_for_seq[tempseq])))).reshape(1, -1)))
		else: # If class features are not to be used
			biGram_features0.append(bigram_features0(prob_pssm))
			biGram_features2.append(bigram_features2(prob_pssm))
			biGram_features3.append(bigram_features3(prob_pssm))
			biGram_features4.append(bigram_features4(prob_pssm))
			biGram_features1.append(bigram_features1(prob_pssm))
			biGram_features5.append(bigram_features5(prob_pssm))

		norm_pssm = prob_pssm + 0.01

		if(len(norm_pssm) < seqlen): # If length of PSSM data is less than seqlen, pad it with zeros
			for j in range(seqlen-len(norm_pssm)): # Pad the data with zeros
				norm_pssm = np.concatenate((norm_pssm,norm_pssm[0]*0))
		else: # If length of PSSM data is greater than seqlen, truncate it
			norm_pssm = norm_pssm[:seqlen]
		pssm.append(norm_pssm) # Append PSSM data to pssm list

# Convert labels, sequences and bigram features to numpy arrays
labels = np.array(labels)
num_classes =  len(np.unique(labels))
foldlabels = pd.get_dummies(labels).values
sequences = np.array(seq)
biGram0 = np.array(biGram_features0)
biGram1 = np.array(biGram_features1)
biGram2 = np.array(biGram_features2)
biGram3 = np.array(biGram_features3)
biGram4 = np.array(biGram_features4)
biGram5 = np.array(biGram_features5)
hmm = np.array(hmm)
pssm = np.array(pssm)

if(rawdata == 'hmm'): # If rawdata is hmm, set matrixdata to hmm
	matrixdata = hmm
else: # If rawdata is pssm, set matrixdata to pssm
	matrixdata = pssm

no_filters1 = 4

In [ ]:
with tf.device('/device:GPU:0'): # Use GPU
	f=0 # Initialize fold count
	config=tf.compat.v1.ConfigProto()
	config.gpu_options.allow_growth = True
	config.gpu_options.per_process_gpu_memory_fraction = 0.2
	tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))		

	acc_k_fold = [] # Initialize list to store accuracies of each fold
	kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42) # Initialize 10-fold cross validation

	for train, test in kf.split(sequences, labels): # Loop through each fold
		f=f+1 # Increment fold count
		X_train, X_test = matrixdata[train], matrixdata[test] # Split matrixdata into training and testing data
		Y_train, Y_test = foldlabels[train], foldlabels[test] # Split foldlabels into training and testing labels
		X_biGram0_Train, X_biGram0_Test = biGram0[train], biGram0[test] # Split biGram0 into training and testing data
		X_biGram1_Train, X_biGram1_Test = biGram1[train], biGram1[test] # Split biGram1 into training and testing data
		X_biGram2_Train, X_biGram2_Test = biGram2[train], biGram2[test] # Split biGram2 into training and testing data
		X_biGram3_Train, X_biGram3_Test = biGram3[train], biGram3[test] # Split biGram3 into training and testing data
		X_biGram4_Train, X_biGram4_Test = biGram4[train], biGram4[test] # Split biGram4 into training and testing data
		X_biGram5_Train, X_biGram5_Test = biGram5[train], biGram5[test] # Split biGram5 into training and testing data

		cnn_input = Input(shape=(seqlen,20), name='cnn_input') # Initialize input layer for CNN
		c_input = Reshape((seqlen,20,1))(cnn_input) # Reshape input layer
		c_output1 = Conv2D(no_filters1, (5,5),  activation='tanh', strides=5, padding='same')(c_input) # Convolution layer
		m_output1 = MaxPooling2D((3,3), strides=3, padding='same')(c_output1) # Max pooling layer
		f_input = Flatten()(m_output1) # Flatten the output of max pooling layer

		bigram_input0 = Input(shape=(X_biGram0_Train.shape[1], X_biGram0_Train.shape[2]), name='bigram_input0') # Initialize input layer for bigram features
		bg0_input = Flatten()(bigram_input0) # Flatten the input layer

		bigram_input1 = Input(shape=(X_biGram1_Train.shape[1], X_biGram1_Train.shape[2]), name='bigram_input1') # Initialize input layer for bigram features
		bg1_input = Flatten()(bigram_input1) # Flatten the input layer

		bigram_input2 = Input(shape=(X_biGram2_Train.shape[1], X_biGram2_Train.shape[2]), name='bigram_input2') # Initialize input layer for bigram features
		bg2_input = Flatten()(bigram_input2) # Flatten the input layer

		bigram_input3 = Input(shape=(X_biGram3_Train.shape[1], X_biGram3_Train.shape[2]), name='bigram_input3') # Initialize input layer for bigram features
		bg3_input = Flatten()(bigram_input3) # Flatten the input layer

		bigram_input4 = Input(shape=(X_biGram4_Train.shape[1], X_biGram4_Train.shape[2]), name='bigram_input4') # Initialize input layer for bigram features
		bg4_input = Flatten()(bigram_input4) # Flatten the input layer

		bigram_input5 = Input(shape=(X_biGram5_Train.shape[1], X_biGram5_Train.shape[2]), name='bigram_input5') # Initialize input layer for bigram features
		bg5_input = Flatten()(bigram_input5) # Flatten the input layer

		bigram_input5 = Input(shape=(X_biGram5_Train.shape[1], X_biGram5_Train.shape[2]), name='bigram_input5') # Initialize input layer for bigram features
		bg5_input = Flatten()(bigram_input5) # Flatten the input layer

		hybrid_features = concatenate([f_input, bg0_input, bg1_input, bg2_input, bg3_input, bg4_input, bg5_input], axis=-1) # Concatenate the features
		d_output2 = Dense(512, activation='tanh')(hybrid_features) # Dense layer
		d_output2 = Dense(128, activation='tanh')(d_output2) # Dense layer
		main_output = Dense(foldlabels.shape[1], activation='softmax', name='main_output', kernel_regularizer=l2(0.01))(d_output2) # Output layer

		# Create model
		model = Model(inputs=[cnn_input, bigram_input0, bigram_input1, bigram_input2, bigram_input3, bigram_input4, bigram_input5], outputs=[main_output])
		model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

		earlyStopping = EarlyStopping(monitor='val_accuracy', patience=50, verbose=0, mode='auto') # early stopper
		load_file = "./model/"+dataset+"_SXG_BiGram_best.h5" # model save file path
		checkpointer = ModelCheckpoint(monitor='val_accuracy', filepath=load_file, verbose=0, save_best_only=True) # checkpointer

		# Fit the model
		history=model.fit({'cnn_input': X_train, 'bigram_input0': X_biGram0_Train, 'bigram_input1': X_biGram1_Train, 'bigram_input2': X_biGram2_Train, 'bigram_input3': X_biGram3_Train, 'bigram_input4': X_biGram4_Train, 'bigram_input5': X_biGram5_Train}, {'main_output': Y_train}, 
			validation_data=({'cnn_input': X_test, 'bigram_input0': X_biGram0_Test, 'bigram_input1': X_biGram1_Test, 'bigram_input2': X_biGram2_Test, 'bigram_input3': X_biGram3_Test, 'bigram_input4': X_biGram4_Test, 'bigram_input5': X_biGram5_Test},{'main_output': Y_test}), 
			epochs=500, batch_size=64, callbacks=[checkpointer, earlyStopping], verbose=0)

		model.load_weights(load_file) # Load the best model and score
		score = model.evaluate({'cnn_input': X_test, 'bigram_input0': X_biGram0_Test, 'bigram_input1': X_biGram1_Test, 'bigram_input2': X_biGram2_Test, 'bigram_input3': X_biGram3_Test, 'bigram_input4': X_biGram4_Test, 'bigram_input5': X_biGram5_Test},{'main_output': Y_test}, verbose=0, batch_size=1)
		print("Fold-",f, ": ", score)

		acc_k_fold.append(score[1]) # Append accuracy of the fold to acc_k_fold list

		# Predict the scores
		pred_scores = model.predict({'cnn_input': X_test, 'bigram_input0': X_biGram0_Test, 'bigram_input1': X_biGram1_Test, 'bigram_input2': X_biGram2_Test, 'bigram_input3': X_biGram3_Test, 'bigram_input4': X_biGram4_Test, 'bigram_input5': X_biGram5_Test})
		print("pred_scores shape =", pred_scores.shape)


	resdata = "Class Features = "+str(clsfeature)+" -- 10-cross fold accuracy of Protein "+predtype+" Prediction of "+dataset+" using "+rawdata+" is :"+str(np.mean(acc_k_fold))+"\n"

	print(resdata) # Print the result
	print("10 Fold Accuracies:", acc_k_fold) # Print the accuracies of each fold

print("# of Labels:", num_classes)
print("# of Labels:", foldlabels.shape[1])
print("hybrid_features count:", hybrid_features.shape)


Fold- 1 :  [1.6671605110168457, 0.8543999791145325]
20/20 [==============================] - 0s 5ms/step
pred_scores shape = (625, 171)
Fold- 2 :  [1.7678273916244507, 0.8416000008583069]
20/20 [==============================] - 0s 5ms/step
pred_scores shape = (625, 171)
Fold- 3 :  [1.8220089673995972, 0.8223999738693237]
20/20 [==============================] - 0s 4ms/step
pred_scores shape = (625, 171)
Fold- 4 :  [1.8152962923049927, 0.8303999900817871]
20/20 [==============================] - 0s 5ms/step
pred_scores shape = (625, 171)
Fold- 5 :  [1.8196617364883423, 0.8191999793052673]
20/20 [==============================] - 0s 5ms/step
pred_scores shape = (625, 171)
Fold- 6 :  [1.718684196472168, 0.8399999737739563]
20/20 [==============================] - 0s 5ms/step
pred_scores shape = (625, 171)
Fold- 7 :  [1.6944830417633057, 0.86080002784729]
20/20 [==============================] - 0s 5ms/step
pred_scores shape = (625, 171)
Fold- 8 :  [1.675512671470642, 0.8543999791145325]